$\displaystyle \mathbf{¿ES ~ CIERTA ~ LA ~ SIGUIENTE ~ AFIRMACION~~}  \int_0^1 \int_0^2 \cdots \int_0^n \sum_{i=1}^n x_i \, dx_1 dx_2 \cdots dx_n = \frac{n(n+1)!}{4}  \mathbf{?} \tag*{}$

$ \mathbf{Luis~Jiménez} \tag*{}$

$ \text{Agosto 2023 - Versión 1.0} \tag*{} $


__6. Apéndice A. Cálculo de la suma (9) con la librería sympy de python:__

Para usar la librería ```sympy``` siempre se define las variables simbólicas con la instrucción ```sympy.symbols()```. Las sumatorias simbólicas se crean mediante ```Sum(sucesión_adentro_de_la_sumatoria, (contador, límite_inferior, límite_superior))```. ```Sum()``` es un objeto que no se ejecuta inmediatamente, se deja  como una sumatoria expresada, para resolverla se invoca el método ```doit()```. El resultado es un polinomio, podemos factorizarlo con la instrucción ```sympy.factor()```. Finalmente, para que se vea bonito el resultado, usamos el comando ```Equality(lado_izquierdo, lado_derecho)``` para crear una ecuación expresada sin evaluar.

In [1]:
import sympy as sym

def Sn(p):
    i, n = sym.symbols('i,n')
    s = sym.Sum(i**p, (i, 1, n))
    return sym.Equality(s, sym.factor(s.doit()))

# puedes poner el argumento p-entero que tú quieras, he puesto aquí 3
Sn(3)

Eq(Sum(i**3, (i, 1, n)), n**2*(n + 1)**2/4)

__7. Apéndice B. Cálculo de los números de Bernoulli:__

En esta sección calculamos los números de Bernoulli con varios métodos. La primera función ```bernoulli_old(n)``` usa la definición recursiva escritas en (14a), (14b). Sin embargo la recursión en python es un procedimiento lento, por eso se utiliza la librería ```functools``` que proporciona el decorador ```@lru_cache``` que aplica la técnica de memoización para guardar resultados en la caché sin tener que volver a invocar a la función, esto hace que se gane bastante rendimiento y por lo tanto se reduzca el tiempo de cómputo drásticamente. Sólo escribimos ```@lru_cache``` en la línea de código encima de la función ```bernoulli_old()``` anterior y la denominamos ```bernoulli(n)```. La tercera función es ```bernoulli_explicit(n)``` que utiliza las fórmulas explícitas de (15). La últma función es ```bernoulli_all(n, method)``` que calcula los números de Bernoulli con las tres funciones anteriores, además con la función ```scipy.special.bernoulli(n)``` disponible en la librería ```scipy``` y a partir de su relación con la función zeta de Riemann de (19). Finalmente calculamos algunos valores variando $n$, calculamos los tiempos de cómputo para comparar y lo tabulamos en un ```pandas.DataFrame```:

In [1]:
import numpy as np
import scipy as sp
from functools import lru_cache
import timeit
import pandas as pd


def bernoulli_old(n, convention=-1):
    if n == 0: return 1
    if n == 1: return 1/2*convention # (por convención)
    if n % 2: return 0  # B(impar) = 0
    s = 1*(convention==1)
    for k in range(n):
        s -= bernoulli_old(k, convention) * np.math.comb(n, k) / (n + 1 - k)
    return s
        
    
@lru_cache
def bernoulli(n, convention=-1):
    if n == 0: return 1
    if n == 1: return 1/2*convention # (por convención)
    if n % 2: return 0  # B(impar) = 0
    s = 1*(convention==1)
    for k in range(n):
        s -= bernoulli(k, convention) * np.math.comb(n, k) / (n + 1 - k)
    return s

def bernoulli_explicit(n, convention=-1):
    X = [(-1)**j*np.math.comb(i, j)*(j + 1*(convention==1))**n/(i + 1) for i in range(n+1) for j in range(i+1)]
    return np.sum(X)


def bernoulli_all(n, method):
    if method=='recursive': return bernoulli_old(n)
    elif method=='recursive+lru_cache': return bernoulli(n)
    elif method=='explicit': return bernoulli_explicit(n)
    elif method=='scipy-bernoulli': return sp.special.bernoulli(n)[-1]
    else: return -n*sp.special.zeta(1-n)
        
number_sim = 100000
n_array = np.arange(2, 13, 2)    
methods = ['recursive', 'recursive+lru_cache','explicit','scipy-bernoulli', 'scipy-zeta']
    
Dic_Bn = {'Bn {}'.format(method):[bernoulli_all(n, method=method) for n in n_array] for method in methods }
Dic_time = {'Time in µs {}'.format(method):[1e6*timeit.timeit(lambda: bernoulli_all(n, method), number=number_sim)/number_sim for n in n_array] for method in methods }
Dic = {'n':n_array, **Dic_Bn, **Dic_time}
df = pd.DataFrame(Dic)
df.set_index('n', inplace=True)
df

C:\Users\User\AppData\Local\Temp\ipykernel_7064\1352221884.py:29: RuntimeWarning: overflow encountered in long_scalars
  X = [(-1)**j*np.math.comb(i, j)*(j + 1*(convention==1))**n/(i + 1) for i in range(n+1) for j in range(i+1)]
C:\Users\User\AppData\Local\Temp\ipykernel_7064\1352221884.py:29: RuntimeWarning: overflow encountered in long_scalars
  X = [(-1)**j*np.math.comb(i, j)*(j + 1*(convention==1))**n/(i + 1) for i in range(n+1) for j in range(i+1)]


,Bn recursive,Bn recursive+lru_cache,Bn explicit,Bn scipy-bernoulli,Bn scipy-zeta,Time in µs recursive,Time in µs recursive+lru_cache,Time in µs explicit,Time in µs scipy-bernoulli,Time in µs scipy-zeta
n,,,,,,,,,,
2,0.166667,0.166667,1.666667e-01,0.166667,0.166667,6.657069,0.487476,19.085522,1.991925,3.186311
4,-0.033333,-0.033333,-3.333333e-02,-0.033333,-0.033333,12.739604,0.463361,31.016918,107.460744,3.741595
6,0.023810,0.023810,2.380952e-02,0.023810,0.023810,18.651163,0.468042,49.772329,105.567953,4.027417
8,-0.033333,-0.033333,-3.333333e-02,-0.033333,-0.033333,35.814148,0.465119,62.955369,114.570888,4.090779
10,0.075758,0.075758,8.676702e+08,0.075758,0.075758,59.177891,0.477088,112.681310,114.319097,4.601215
12,-0.253114,-0.253114,-2.643295e+08,-0.253114,-0.253114,118.075945,0.475995,170.593817,106.954235,4.233016


De acuerdo a la tabla anterior, la fórmula explícita resulta en errores de redondeo a partir de $n=10$, así que es más confiable utilizar la versión recursiva. El tiempo de cómputo en recursivas duplica con $n$, sin ambargo cuando se utiliza memoización con ```@lru_cache``` se realizan los cálculos en un tiempo constante de 0.5 microsegundos, unas doscientas veces más rápido que utilizar la función de la librería scipy, y es 8 veces más rápido que utilizando la función zeta de Riemann de esta misma librería. Concluimos pr acá que el procedimiento recursivo con el decorador ```@lru_cache``` es la mejor forma de calcular los números de Bernoulli con python.

__8. Apéndice C. Cálculo de la integral (1) mediante el método de Montecarlo:__

El método de Montecarlo es muy fácil de programar, sólo hay que generar ```n``` números pseudoaleatorios y guardarlos en un array llamado ```X```, calcular ```Z``` con la Ecuación (21), donde la potencia ```p``` también es un argumento de la función. Este procedimiento lo repetimos ```N_montecarlo``` veces, típicamente, un millón de veces, mientras más intentos mejor, mayor precisión tendrán los resultados. Luego se toma el promedio aritmético de todos los resultados ```Z``` y se multiplica por $n!$ para obtener el valor de la integral. Se comparan los resultados con la fórmula exacta (13). Más abajo se muestra la tabla variando $n$ y $p$, y podemos observar que las aproximaciones con el método de Montecarlo son muy buenas acertando 3-4 cifras significativas con ```N_montecarlo=1e6```.

In [61]:

def multiple_integral(n, p, N_montecarlo=int(1e6), method='montecarlo'):
    n_fact = np.math.factorial(n)
    r = n_fact/(p + 1)
    if method=='montecarlo':
        V = n_fact
        Z = np.zeros(N_montecarlo, dtype=float)
        for i in range(N_montecarlo):
            X = np.array( [((n - k)*np.random.rand()) for k in range(n)] )
            Z[i] = sum(X**p)
        return np.mean(Z)*V

    elif method=='exact':
        s = sum([i**p for i in range(n+1)])
        return r*s
    else:
        P = p + 1
        s = 1/P*np.sum([np.math.comb(P, i)*bernoulli(i, convention=1)*n**(P-i) for i in range(P)])
        return r*s
    
n_array = [1, 2, 3, 5, 10, 12]
p_array = [1, 2, 5]
methods = ['montecarlo', 'Faulhaber (exact)']
Dic = { '{}, p={}'.format(method, p):[multiple_integral(n, p, method=method) for n in n_array]  for p in p_array for method in methods}
Dic['n'] = n_array
df = pd.DataFrame(Dic)
df.set_index('n', inplace=True)
df

,"montecarlo, p=1","Faulhaber (exact), p=1","montecarlo, p=2","Faulhaber (exact), p=2","montecarlo, p=5","Faulhaber (exact), p=5"
n,,,,,,
1,4.997534e-01,5.000000e-01,3.334338e-01,3.333333e-01,1.667907e-01,1.666667e-01
2,2.998458e+00,3.000000e+00,3.333369e+00,3.333333e+00,1.099564e+01,1.100000e+01
3,1.799120e+01,1.800000e+01,2.800687e+01,2.800000e+01,2.760692e+02,2.760000e+02
5,9.001550e+02,9.000000e+02,2.198885e+03,2.200000e+03,8.847959e+04,8.850000e+04
10,9.977799e+07,9.979200e+07,4.655645e+08,4.656960e+08,1.336242e+11,1.335550e+11
12,1.868031e+10,1.868106e+10,1.037267e+11,1.037837e+11,5.038882e+13,5.035169e+13


__9. Apéndice D. Cálculo de la distancia promedio mediante el método de Montecarlo para varios valores de $n$:__

La función ```generate_randompoints(n, D, shape)``` genera un vector de ```n``` variables pseudoaleatorias en el intervalo $[-D/2,D/2]$ con densidad de probabilidad uniforme, ```shape``` es el parámetro que modifica la forma geométrica por medio de la norma $\| ~~.~ \|_{\rho}$, se introduce ```shape=numpy.inf``` si es un hipercubo y ```shape=2``` si es una hiperesfera. La función ```expected_distance(n, D, shape, N_montecarlo)``` calcula la distancia promedio con el método de Montecarlo, siendo ```N_montecarlo``` el número de simulaciones. Con la finalidad de comparar los valores anteriores, se escribe la función ```expected_distance_exact(n, D, name='n-ball')``` que calcula el valor exacto con las expresiones (25)-(31a) y finalmente se tabulan los resultados. Correr este código toma alrededor de DOS HORAS dependiendo de la máquina, python es lentísimo para esto, se hubiese podido realizar una paralelización distribuyendo las tareas en varios cores, pero bueno, esto es lo que hay por los momentos.

In [55]:
def generate_randompoints(n, D, shape):
    R = D/2
    repeat = True
    while repeat:
        X = np.random.uniform(low=-R, high=R, size=(1, n))
        repeat = np.linalg.norm(X, ord=shape) > R
        if np.isinf(shape): repeat = False
    return X


def expected_distance(n=3, D=1, shape=2, N_montecarlo=int(1e5)):
    s = 0   
    for i in range(N_montecarlo):
        X = generate_randompoints(n, D, shape)
        Y = generate_randompoints(n, D, shape)
        s += np.linalg.norm(X - Y)
    return s / N_montecarlo


def expected_distance_exact(n=3, D=1, name='n-ball'):
    factorial = lambda n: sp.special.gamma(n + 1)
    ln = lambda x: np.log(x)
    
    if name=='n-ball':
        E = n/(2*n + 1)
        if n % 2 == 0:
            b = (3*n + 1)*ln(2) + 2*ln(factorial(n/2)) + ln(factorial(n)) - \
                ln(n + 1) - ln(factorial(2*n)) - ln(np.pi)
        else:
            b = (n + 1)*ln(2) + 3*ln(factorial(n)) - ln(n + 1) - \
               2*ln(factorial((n - 1)/2)) - ln(factorial(2*n))
            
        E *= np.exp(b)
        
    else:
        if n==1: E = 1/3
        elif n==2: E = (2 + np.sqrt(2) + 5*np.log(1 + np.sqrt(2)))/15
        elif n==3: 
            a = (4 + 17*np.sqrt(2) - 6*np.sqrt(3) - 7*np.pi)/105
            b = np.log(1 + np.sqrt(2))/5
            c = 2*np.log(2 + np.sqrt(3))/5
            E = a + b + c
        else:
            Emin = np.sqrt(n)/3
            Emax = np.sqrt(n/6)*np.sqrt((1 + 2*np.sqrt(1 - 3/(5*n)))/3)
            E = np.array([Emin, Emax])
    E *= D
    return E
            

n_array = np.arange(1, 11, dtype=int)
q_array = np.array([np.inf, 2])
name_array = ['n-hipercube', 'n-ball']

Dic1 = { '{} (montecarlo)'.format(name):[expected_distance(n, shape=q) for n in n_array]  for q, name in zip(q_array, name_array) }
Dic2 = { '{} (exact)'.format(name):[expected_distance_exact(n, name=name) for n in n_array]  for name in name_array }
Dic = {'n':n_array, **Dic1, **Dic2}
df = pd.DataFrame(Dic)
df.set_index('n', inplace=True)
df


,n-hipercube (montecarlo),n-ball (montecarlo),n-hipercube (exact),n-ball (exact)
n,,,,
1,0.332557,0.333856,0.333333,0.333333
2,0.522163,0.452254,0.521405,0.452707
3,0.661914,0.514787,0.661707,0.514286
4,0.775971,0.551505,"[0.6666666666666666, 0.794971542671473]",0.551872
5,0.878839,0.577484,"[0.7453559924999299, 0.8938316868543125]",0.577201
6,0.968305,0.595371,"[0.8164965809277259, 0.9827455750940877]",0.595426
7,1.051203,0.609269,"[0.8819171036881969, 1.0642305625439028]",0.609169
8,1.128325,0.620264,"[0.9428090415820635, 1.1398905222612163]",0.619901
9,1.199441,0.628556,"[1.0, 1.2108227711268464]",0.628513
